In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secret_label = "HF Hub"
secret_value = "###"
login(token=secret_value)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Installations 

In [2]:
!pip install accelerate transformers wandb einops datasets peft wandb bitsandbytes trl git+https://github.com/huggingface/peft.git evaluate rouge_score loralib

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ntcvnn7a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ntcvnn7a
  Resolved https://github.com/huggingface/peft.git to commit 58afb34ea03ae1166fc331adee0b4646e46db6dc
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.3 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.11.2.dev0-py3-none-any.whl size=270929 sha256=

In [3]:
import wandb
import os

os.environ['WANDB_API_KEY'] = '###'
wandb.login()

wandb: Currently logged in as: daniteebee. Use `wandb login --relogin` to force relogin


True

In [4]:
import torch
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
import pandas as pd
import evaluate
from string import Template

2024-07-08 22:18:12.972063: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 22:18:12.972193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 22:18:13.109150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline,logging,Trainer
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [6]:
import transformers

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [7]:
# def print_number_of_trainable_model_parameters(model):
#     trainable_model_params = 0
#     all_model_params = 0
#     for _, param in model.named_parameters():
#         all_model_params += param.numel()
#         if param.requires_grad:
#             trainable_model_params += param.numel()
#     return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# print(print_number_of_trainable_model_parameters(model))

# Load 3 datasets

In [8]:
from datasets import load_dataset

dataset1 = load_dataset("Amod/mental_health_counseling_conversations")
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")
dataset3 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

In [9]:
df1= dataset1["train"].data.to_pandas().dropna()
df2= dataset2["train"].data.to_pandas().dropna()
df3= dataset3["train"].data.to_pandas().dropna()

# Preprocess datasets

In [10]:
df1.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [11]:
df2.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,text
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971,<s>[INST] I have so many issues to address. I ...
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386,<s>[INST] I have so many issues to address. I ...
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071,<s>[INST] I have so many issues to address. I ...
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643,<s>[INST] I have so many issues to address. I ...
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256,<s>[INST] I have so many issues to address. I ...


In [12]:
# Just rename columns
df2 = df2[["questionText", "answerText"]]
df2.columns = ["question", "answer"]
df2

,question,answer
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i..."
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...
...,...,...
2770,What are some difficulties that a counselor ca...,Although many clients have the capacity to be ...
2771,What are some difficulties that a counselor ca...,"I usually don't label a client as ""difficult"" ..."
2772,What are some difficulties that a counselor ca...,"Dang right! :)Heh heh, and correct me if I'm ..."
2773,What are some difficulties that a counselor ca...,"Yes, just like some relationships outside of o..."


In [13]:
df3.head()

,text
0,<HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1,<HUMAN>: What are symptoms of panic attack vs....
2,<HUMAN>: What are the types of Mental Illness?...
3,<HUMAN>: What does mental-illness mean?\n<ASSI...
4,<HUMAN>: How can you treat mental illness?\n<A...


In [14]:
# Split single sample into Question & Answer

def split_qa(text):
    # below split returns ["", "question\n", "answer "]
    question, answer = re.split(r" ?<\w+>: ?", text)[1:]
    question = question.strip().replace("\n","")
    answer = answer.strip().replace("\n", "")
    return question, answer
    
df3["text"] = df3["text"].apply(split_qa)
df3[["question", "answer"]] = pd.DataFrame(data=[[q, a] for q, a in df3["text"].values])
df3.drop(columns=["text"], inplace=True)
df3


,question,answer
0,What is a panic attack?,Panic attacks come on suddenly and involve int...
1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,What are the types of Mental Illness?,There are many different conditions that are r...
3,What does mental-illness mean?,Mental illness is a group of biological brain ...
4,How can you treat mental illness?,The treatment of mental illness is a multi-fac...
...,...,...
167,Who are you?,Hello there! I am your empathetic Mental Healt...
168,What is the helpline number for suicide preven...,I'm really sorry but I can't provide helpline ...
169,Which is the best mental health hospital in Ne...,"I'm really sorry, but I can't provide accurate..."
170,Who is the best hypnotherapist in London?,I understand that you're looking for a hypnoth...


In [15]:
# Merge datasets
df = pd.concat((df1, df2, df3)).sample(frac=1)
df

,Context,Response,question,answer
3045,I need to get on base to see my doctor. My ID ...,You should be able to get a replacement ID car...,NaN,NaN
1211,I’m in a relationship with my child's father. ...,It may be important to look at this as getting...,NaN,NaN
2023,NaN,NaN,We've been in a long distance relationship for...,The best way to get an answer is to just ask. ...
2835,She works with a man and always looks at guys ...,I wonder if you are having some of these thoug...,NaN,NaN
1533,NaN,NaN,I crossdress and like to be feminine but I am ...,Thanks for asking this question. I think the m...
...,...,...,...,...
3449,I've gone to a couple therapy sessions so far ...,It can be uncomfortable talking to someone abo...,NaN,NaN
1425,My mom is always bossy and treats me like a ch...,"This is a common issue. You grow up gradually,...",NaN,NaN
33,NaN,NaN,Why do some people choose to just take meds an...,It's understandable that some people choose to...
159,NaN,NaN,What are some resources or support groups I ca...,There are various resources and support groups...


In [16]:
template = Template('<HUMAN>: $human\n<ASSISTANT>: $assistant')
def format_input(df, idx):
    
    human = df.loc[idx, 'question']
    assistant = df.loc[idx, 'answer']
   

    input_text = template.substitute(
        human=human, assistant=assistant)
    
    return input_text

In [17]:
index = [i for i in range(df.shape[0])]
df_red = df.reset_index(drop=True)
df_index = df_red.reindex(index)
df_index

,Context,Response,question,answer
0,I need to get on base to see my doctor. My ID ...,You should be able to get a replacement ID car...,NaN,NaN
1,I’m in a relationship with my child's father. ...,It may be important to look at this as getting...,NaN,NaN
2,NaN,NaN,We've been in a long distance relationship for...,The best way to get an answer is to just ask. ...
3,She works with a man and always looks at guys ...,I wonder if you are having some of these thoug...,NaN,NaN
4,NaN,NaN,I crossdress and like to be feminine but I am ...,Thanks for asking this question. I think the m...
...,...,...,...,...
6291,I've gone to a couple therapy sessions so far ...,It can be uncomfortable talking to someone abo...,NaN,NaN
6292,My mom is always bossy and treats me like a ch...,"This is a common issue. You grow up gradually,...",NaN,NaN
6293,NaN,NaN,Why do some people choose to just take meds an...,It's understandable that some people choose to...
6294,NaN,NaN,What are some resources or support groups I ca...,There are various resources and support groups...


In [18]:
for i in range(df_index.shape[0]):
    df_index.loc[i, 'text'] = format_input(df_index, i)

In [19]:
df = df_index

In [20]:
# Remove texts and keep input_ids and labels
df.drop(columns=["question", "answer"], inplace=True)

In [21]:
df['text'][0]

'<HUMAN>: nan\n<ASSISTANT>: nan'

In [22]:
df

,Context,Response,text
0,I need to get on base to see my doctor. My ID ...,You should be able to get a replacement ID car...,<HUMAN>: nan\n<ASSISTANT>: nan
1,I’m in a relationship with my child's father. ...,It may be important to look at this as getting...,<HUMAN>: nan\n<ASSISTANT>: nan
2,NaN,NaN,<HUMAN>: We've been in a long distance relatio...
3,She works with a man and always looks at guys ...,I wonder if you are having some of these thoug...,<HUMAN>: nan\n<ASSISTANT>: nan
4,NaN,NaN,<HUMAN>: I crossdress and like to be feminine ...
...,...,...,...
6291,I've gone to a couple therapy sessions so far ...,It can be uncomfortable talking to someone abo...,<HUMAN>: nan\n<ASSISTANT>: nan
6292,My mom is always bossy and treats me like a ch...,"This is a common issue. You grow up gradually,...",<HUMAN>: nan\n<ASSISTANT>: nan
6293,NaN,NaN,<HUMAN>: Why do some people choose to just tak...
6294,NaN,NaN,<HUMAN>: What are some resources or support gr...


In [23]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [24]:
# Split data into train & test
# train_df, test_df = train_test_split(tokenized_df, test_size=0.15)

# print(f"Training Size: {train_df.shape[0]} samples")
# print(f"Test Size: {test_df.shape[0]} samples")

In [25]:
train_ratio = 0.6
train_size = int(len(df) * train_ratio)

train_dataset = df[:train_size]
eval_dataset = df[train_size:]

In [26]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)
train_dataset = train_dataset.remove_columns('__index_level_0__')
eval_dataset = eval_dataset.remove_columns('__index_level_0__')
# train_dataset = Dataset.from_pandas(df)
# train_dataset = train_dataset.remove_columns('__index_level_0__')

In [27]:
train_dataset

Dataset({
    features: ['Context', 'Response', 'text'],
    num_rows: 3777
})

# Configure PEFT Trainer

In [28]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [29]:
from transformers import AutoTokenizer
from peft import prepare_model_for_kbit_training


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config=quant_config,
    device_map="auto",
)
model.config.use_cache = False
# model.config.pretraining_tp = 1

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [30]:
# Set pad_token as end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [31]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.05
lora_r = 64


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [32]:
model = get_peft_model(model, peft_config) 
model.print_trainable_parameters()

trainable params: 27,262,976 || all params: 8,057,524,224 || trainable%: 0.3384


In [33]:
output_dir = "./fined_results"
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 16
optim = "paged_adamw_32bit"
save_steps = 500
num_train_epochs = 1
logging_steps = 100
learning_rate = 6e-5
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
a = ['linear', 'cosine', 'cosine_with_restarts', 'polynomial',\
     'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau', 'cosine_with_min_lr', 'warmup_stable_decay']
lr_scheduler_type = a[4]
     

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    bf16=False,
    max_grad_norm=max_grad_norm,
#     max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [34]:
from trl import SFTTrainer

max_seq_length = 512
# 
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# torch.cuda.empty_cache()
# trainer.train()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3777 [00:00<?, ? examples/s]

Map:   0%|          | 0/2519 [00:00<?, ? examples/s]

# Training

In [35]:
torch.cuda.empty_cache()

In [36]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: wandb version 0.17.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240708_222056-8ff3xsyg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./fined_results
wandb: ⭐️ View project at https://wandb.ai/daniteebee/huggingface
wandb: 🚀 View run at https://wandb.ai/daniteebee/huggingface/runs/8ff3xsyg
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recom

Step,Training Loss


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# Save model & Publish

In [ ]:
trainer.model.save_pretrained("./llama-3-8B-actual-therapist")
tokenizer.save_pretrained("./llama-3-8B-actual-therapist")

In [ ]:
# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./llama-3-8B-actual-therapist", quantization_config=quant_config).to(device)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./llama-3-8B-actual-therapist")

In [ ]:
# model.push_to_hub("EYEDOL/LLAMA_3_MODEL_THERAPIST")
# tokenizer.push_to_hub("EYEDOL/LLAMA_3_MODEL_THERAPIST")

# Inference

In [ ]:
def single_inference(question):
    messages = [
        {"role": "system", "content": "Reply to the following inquiry."},
    ]

    messages.append({"role": "user", "content": question})


    input_ids = fine_tuned_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        fine_tuned_tokenizer.eos_token_id,
        fine_tuned_tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = fine_tuned_model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.01,
    #     top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    output = fine_tuned_tokenizer.decode(response, skip_special_tokens=True)
    return output

In [ ]:
question = """I feel like I don't exist and my body is not my own, I dont just know what is wrong. what could be wrong with me?"""

answer = single_inference(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')

In [ ]:
question = """I'm always confused when making decisions, I cannot choose one option, what are the causes of being
indecisive?"""

answer = single_inference(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')